## Object selectivity (t-statistics)
This code examines the statistical significance of beta coefficients based on t-statistics.

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter

import matplotlib as mpl
import matplotlib.pyplot as plt  
import matplotlib.patches as mpatches

from datetime import date
import time

from joblib import Parallel, delayed

import h5py

In [2]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [3]:
mother_path = Path('D:/Multi-modal project/')

### Parameter setting

In [4]:
alpha = 0.01
cons_bin_crit = 5

gauss_sigma = 2

# colors for multimodal, vis-only, aud-only conditions
color = ['mediumorchid','cornflowerblue','lightcoral','gray']
color2 = ['cyan','magenta','black']
linestyle = ['-',':']

today = str(date.today())

### Data preparation

In [5]:
save_path = mother_path /'analysis'/'result'/'4. Object selectivity'/today
cell_path = mother_path/'analysis'/'result'/'zFR export'/'13-Apr-2022 (5 trials)'
data_path = mother_path /'analysis'/'result'/'3. Multiple linear regression'

cell_list = os.listdir(cell_path)

# load hdf5 files containing shuffled results
f = h5py.File(data_path/'2023-05-17'/'2023-05-17_multiple_regression_result.hdf5','r')

# make hdf5 save file
os.makedirs(save_path,exist_ok=True)
#s = h5py.File(save_path/f'{today}_object_selectivity_result.hdf5','w')

# Data analysis

In [6]:
def consecutive_bin(sig_bin,cons_bin_crit):
    diff = np.diff(sig_bin)
    array_split = np.split(sig_bin,np.where(diff!=1)[0]+1)
    filtered = [a for a in array_split if len(a)>=cons_bin_crit]
    if len(filtered)!=0:
        result = np.concatenate(filtered)    
    else:
        result = np.array([])
    return result

In [7]:
def plot_SDF_beta(df,linewidth,smooth,save,save_format):
    """
    This function plots mean firing rate patterns of each stimulus condition
    and beta coefficients for visual and auditory terms in multiple linear regression.
    """
    cond = [(df.Type=='Multimodal')&(df.RWD_Loc==boy_goal),
            (df.Type=='Multimodal')&(df.RWD_Loc==egg_goal),
            (df.Type=='Visual')&(df.RWD_Loc==boy_goal),
            (df.Type=='Visual')&(df.RWD_Loc==egg_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==boy_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==egg_goal)]
    
    cell_full_name = cell_name.strip('.csv')
    
    fr_mean = np.zeros((len(cond),95))
    fr_sem = np.zeros((len(cond),95))
    for i in range(len(cond)):
        fr_mean[i,:] = df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy().mean(axis=0)
        fr_sem[i,:] = stats.sem(df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy())
    
    if smooth:
        for i in range(len(cond)):
            fr_mean[i,:] = gaussian_filter(fr_mean[i,:],sigma=gauss_sigma)
            fr_sem[i,:] = gaussian_filter(fr_sem[i,:],sigma=gauss_sigma)
            
    y_max = np.ceil(np.max(fr_mean+fr_sem))
    y_min = np.ceil(np.abs(np.min(fr_mean-fr_sem)))*-1
    
    fig,ax = plt.subplots(3,4,figsize=(20,10))
    plt.suptitle(cell_full_name,fontsize=15);
    x = np.arange(95)*10
    
    for i in range(len(cond)):
        if i%2==0:
            ls = linestyle[0]
        else:
            ls = linestyle[1]            
        ax[0,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[0,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        ax[i%2+1,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[i%2+1,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        if i<6:
            ax[int(np.floor(i/2)),1].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
            ax[int(np.floor(i/2)),1].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
    
    # control stimulus SDF
    #for i in range(3):
    #    ax[i,1].plot(x,fr_mean[6,:],color=color[3])
    #    ax[i,1].plot(x,fr_mean[7,:],color=color[3],linestyle=':')
    
    # SDF for each condition
    t1 = ['All conditions','Boy condition','Egg condition','Multimodal','Visual-only','Auditory-only']
    for i in range(6):
        #ax[i%3,int(np.floor(i/3))].scatter(visual_bin*10,np.tile(y_max-0.1,(len(visual_bin),1)),
        #                                   color=color2[0],marker='*',s=5)
        #ax[i%3,int(np.floor(i/3))].scatter(auditory_bin*10,np.tile(y_max-0.3,(len(auditory_bin),1)),
        #                                   color=color2[1],marker='*',s=5)
        ax[i%3,int(np.floor(i/3))].set_title(t1[i],fontsize=13)
        ax[i%3,int(np.floor(i/3))].set_yticks(np.arange(y_min,y_max+0.1,1))
        ax[i%3,int(np.floor(i/3))].set_ylabel('z-scored FR',fontsize=13)
        ax[i%3,int(np.floor(i/3))].set_xlabel('Time (ms)',fontsize=13)  
        ax[i%3,int(np.floor(i/3))].set_xticks([0,400,950])
        ax[i%3,int(np.floor(i/3))].set_xlim([0,950])    
    
    if smooth:
        for i in range(len(beta_coef)):
            beta_coef[i,:] = gaussian_filter(beta_coef[i,:],sigma=gauss_sigma)
    
    beta_max = np.ceil(2*np.max(beta_coef))*0.5
    beta_min = np.ceil(2*np.abs(np.min(beta_coef)))*-0.5
    
    # beta coefficient plot
    t2 = ['Boy image','Boy sound','Boy interaction','Egg image','Egg sound','Egg interaction']
    for i in range(6):
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].plot(x,beta_coef[i,:],
                                                 color=color2[divmod(i,3)[1]],
                                                 linestyle='-',linewidth=linewidth)
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].scatter(sig_bin[i][0]*10,
                                                    np.tile(beta_max-0.1,(len(sig_bin[i][0]),1)),
                                                    color=color2[divmod(i,3)[1]],marker='*',s=5)

        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_title(t2[i],fontsize=13)
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_yticks(np.arange(beta_min,beta_max+0.1,0.5))    
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_ylabel('Beta coefficient',fontsize=13)
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_xlabel('Time (ms)',fontsize=13)    
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_xticks([0,400,950])
        ax[divmod(i,3)[1],divmod(i,3)[0]+2].set_xlim([0,950])
        
        ax2 = ax[divmod(i,3)[1],divmod(i,3)[0]+2].twinx()
        ax2.plot(x,pval_log[i,:],color='gray')
        ax2.set_ylabel('-log(pval)')
        ax2.set_yticks(np.arange(-2,17,2))
        ax2.set_ylim([-2,16])
        ax2.axhline(y=-1*np.log(0.05),linestyle=':',color='red')
        ax2.axhline(y=-1*np.log(0.01),linestyle=':',color='red')
        ax2.axhline(y=-1*np.log(0.001),linestyle=':',color='red')        
        
    plt.tight_layout()
    
    if save:
        fig_path = save_path / region
        os.makedirs(fig_path, exist_ok=True)

        if save_format == 'png':
            plt.savefig(fig_path / f'{cell_full_name}.png', dpi=100, facecolor='white')
        elif save_format == 'svg':
            plt.savefig(fig_path / f'{cell_full_name}.svg')
        plt.close()

In [8]:
def save_result(f):
    """
    This function saves regression results into HDF5 format.
    """
    cell_group = f.create_group(str(cell_id))
    
    cell_group.create_dataset('visual_coef',data=visual)
    cell_group.create_dataset('auditory_coef',data=auditory)   
    cell_group.create_dataset('visual_crit',data=visual_crit)
    cell_group.create_dataset('auditory_crit',data=auditory_crit)
    cell_group.create_dataset('visual_bin',data=visual_bin)
    cell_group.create_dataset('auditory_bin',data=auditory_bin)
    cell_group.create_dataset('visual_peak',data=visual_peak)
    cell_group.create_dataset('auditory_peak',data=auditory_peak)
    
    cell_group.attrs['Rat'] = rat_id
    cell_group.attrs['Region'] = region
    cell_group.attrs['Session'] = session_id
    cell_group.attrs['Category'] = cat

In [ ]:
%%time
for cell_run,cell_name in enumerate(cell_list):
    loop_start = time.time()
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id, rat_id, session_id, region = int(cell_info[0]), cell_info[1], cell_info[2], cell_info[5]
    
    # skip response-selective cell
    if f[str(cell_id)].attrs['Response cell'] == 1:
        continue
    
    # load cell data
    df = pd.read_csv(cell_path/cell_name)
    df.drop(df[df.Correctness==0].index,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df[['Visual','Auditory']] = df[['Visual','Auditory']].fillna('no')
    
    boy_goal = df.loc[df['Visual']=='Boy','RWD_Loc'].values[0]
    boy_aud = df.loc[df['RWD_Loc']==boy_goal,'Auditory'].values[0]
    
    egg_goal = df.loc[df['Visual']=='Egg','RWD_Loc'].values[0]
    egg_aud = df.loc[df['RWD_Loc']==egg_goal,'Auditory'].values[0]
    
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    
    # load regression results
    beta_coef = np.array(f[f'{cell_id}/basic/beta_coef'])
    pval = np.array(f[f'{cell_id}/basic/pval'])    
    pval_log = -1*np.log(pval)        
    
    sig_bin = [np.where(pval[i,:]<alpha) for i in range(6)]
    
    # plot SDF and beta coefficient + mark bins with significant selectivity    
    plot_SDF_beta(df,2,1,1,'png')
    
    # save results into HDF5 format
    #save_result(s)    
                
    loop_end = time.time()
    loop_time = divmod(loop_end-loop_start,60)
    print(cell_name.strip('.csv'), f'////// {cell_run+1}/{len(cell_list)} completed  //////  {int(loop_time[0])} min {loop_time[1]:.2f} sec')

0003-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.1 ////// 1/878 completed  //////  0 min 1.26 sec
0004-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.2 ////// 2/878 completed  //////  0 min 1.41 sec
0005-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.3 ////// 3/878 completed  //////  0 min 1.52 sec
0006-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.1 ////// 4/878 completed  //////  0 min 1.53 sec
0007-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.2 ////// 5/878 completed  //////  0 min 2.19 sec
0008-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.3 ////// 6/878 completed  //////  0 min 1.40 sec
0010-600-1-1-Crossmodal-TeV-deep-(-6.96 mm)-TT6.2 ////// 7/878 completed  //////  0 min 1.76 sec
0011-600-1-1-Crossmodal-PER-superficial-(-6.48 mm)-TT7.1 ////// 8/878 completed  //////  0 min 1.30 sec
0013-600-1-1-Crossmodal-PER-deep-(-6.48 mm)-TT8.2 ////// 9/878 completed  //////  0 min 1.40 sec
0014-600-1-1-Crossmodal-PER-deep-(-6.48 mm)-TT8.3 ////// 10/878 completed  //////  0 min 1.52 sec
0015

0176-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.11 ////// 88/878 completed  //////  0 min 1.68 sec
0177-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.2 ////// 89/878 completed  //////  0 min 1.36 sec
0178-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.3 ////// 90/878 completed  //////  0 min 1.47 sec
0179-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.4 ////// 91/878 completed  //////  0 min 1.39 sec
0181-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.6 ////// 92/878 completed  //////  0 min 1.58 sec
0182-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.7 ////// 93/878 completed  //////  0 min 1.41 sec
0183-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.8 ////// 94/878 completed  //////  0 min 1.46 sec
0184-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.9 ////// 95/878 completed  //////  0 min 1.50 sec
0185-602-1-1-Crossmodal-PER-deep-(-4.8 mm)-TT7.1 ////// 96/878 completed  //////  0 min 3.45 sec
0188-602-1-1-Crossmodal-TeV-deep-(-4.44 mm)-TT9.2 ////// 97/878 completed  //////  0 min 1.60 sec
0190-602-1-1-Crossmo

0299-602-3-3-Crossmodal-Auditory-superficial-(-5.52 mm)-TT19.5 ////// 173/878 completed  //////  0 min 2.49 sec
0304-602-3-3-Crossmodal-TeV-superficial-(-5.64 mm)-TT21.4 ////// 174/878 completed  //////  0 min 2.19 sec
0307-602-3-3-Crossmodal-PER-superficial-(-5.4 mm)-TT22.2 ////// 175/878 completed  //////  0 min 2.15 sec
0308-602-3-3-Crossmodal-PER-superficial-(-5.4 mm)-TT22.3 ////// 176/878 completed  //////  0 min 1.98 sec
0309-602-3-3-Crossmodal-PER-superficial-(-5.4 mm)-TT22.4 ////// 177/878 completed  //////  0 min 2.08 sec
0311-602-4-4-Crossmodal-PER-superficial-(-5.28 mm)-TT6.1 ////// 178/878 completed  //////  0 min 2.56 sec
0312-602-4-4-Crossmodal-PER-superficial-(-5.28 mm)-TT6.2 ////// 179/878 completed  //////  0 min 2.38 sec
0313-602-4-4-Crossmodal-PER-superficial-(-5.28 mm)-TT6.3 ////// 180/878 completed  //////  0 min 2.54 sec
0314-602-4-4-Crossmodal-PER-deep-(-4.8 mm)-TT7.1 ////// 181/878 completed  //////  0 min 3.00 sec
0315-602-4-4-Crossmodal-PER-deep-(-4.8 mm)-TT7.

1050-640-3-3-Crossmodal-PER-deep-(-5.52 mm)-TT5.1 ////// 264/878 completed  //////  0 min 2.17 sec
1051-640-3-3-Crossmodal-PER-deep-(-4.8 mm)-TT10.1 ////// 265/878 completed  //////  0 min 2.54 sec
1052-640-3-3-Crossmodal-PER-deep-(-4.8 mm)-TT10.2 ////// 266/878 completed  //////  0 min 2.49 sec
1056-640-3-3-Crossmodal-PER-deep-(-4.56 mm)-TT11.2 ////// 270/878 completed  //////  0 min 2.51 sec
1057-640-3-3-Crossmodal-PER-deep-(-4.56 mm)-TT11.3 ////// 271/878 completed  //////  0 min 2.76 sec
1058-640-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT13.1 ////// 272/878 completed  //////  0 min 3.34 sec
1059-640-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT13.2 ////// 273/878 completed  //////  0 min 2.58 sec
1062-640-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT15.2 ////// 275/878 completed  //////  0 min 2.46 sec
1063-640-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT15.3 ////// 276/878 completed  //////  0 min 2.62 sec
1067-640-3-3-Crossmodal-TeV-deep-(-5.88 mm)-TT22.1 ////// 277/878 completed

1385-647-6-6-Crossmodal-PER-deep-(-5.4 mm)-TT9.3 ////// 356/878 completed  //////  0 min 3.10 sec
1391-647-6-6-Crossmodal-PER-deep-(-5.2 mm)-TT15.4 ////// 357/878 completed  //////  0 min 3.13 sec
1392-647-6-6-Crossmodal-PER-deep-(-5.2 mm)-TT15.5 ////// 358/878 completed  //////  0 min 3.23 sec
1394-647-6-6-Crossmodal-TeV-superficial-(-5.52 mm)-TT16.2 ////// 360/878 completed  //////  0 min 3.64 sec
1395-647-6-6-Crossmodal-TeV-superficial-(-5.52 mm)-TT16.3 ////// 361/878 completed  //////  0 min 2.70 sec
1396-647-6-6-Crossmodal-TeV-superficial-(-5.52 mm)-TT16.4 ////// 362/878 completed  //////  0 min 2.60 sec
1399-647-6-6-Crossmodal-PER-superficial-(-5.88 mm)-TT19.1 ////// 363/878 completed  //////  0 min 2.54 sec
1401-647-6-6-Crossmodal-PER-superficial-(-5.88 mm)-TT19.3 ////// 364/878 completed  //////  0 min 3.12 sec
1404-647-6-6-Crossmodal-PER-superficial-(-6.24 mm)-TT20.3 ////// 365/878 completed  //////  0 min 4.05 sec
1406-647-6-6-Crossmodal-TeV-deep-(-6.12 mm)-TT21.2 ////// 366/

1646-654-5-5-Crossmodal-PER-superficial-(-5.52 mm)-TT11.1 ////// 456/878 completed  //////  0 min 3.58 sec
1651-654-5-5-Crossmodal-Auditory-deep-(-4.8 mm)-TT14.1 ////// 458/878 completed  //////  0 min 4.97 sec
1653-654-5-5-Crossmodal-Auditory-deep-(-5.2 mm)-TT16.2 ////// 460/878 completed  //////  0 min 3.42 sec
1654-654-5-5-Crossmodal-Auditory-deep-(-5.2 mm)-TT16.3 ////// 461/878 completed  //////  0 min 3.32 sec
1655-654-5-5-Crossmodal-Tev-deep-(-5.88 mm)-TT18.1 ////// 462/878 completed  //////  0 min 3.47 sec
1656-654-5-5-Crossmodal-Tev-deep-(-5.88 mm)-TT18.2 ////// 463/878 completed  //////  0 min 6.14 sec
1657-654-5-5-Crossmodal-TeV-deep-(-6.12 mm)-TT21.1 ////// 464/878 completed  //////  0 min 3.27 sec
1659-654-5-5-Crossmodal-TeV-deep-(-6.12 mm)-TT21.3 ////// 465/878 completed  //////  0 min 3.29 sec
1660-654-5-5-Crossmodal-TeV-deep-(-6.12 mm)-TT21.4 ////// 466/878 completed  //////  0 min 3.14 sec
1661-654-5-5-Crossmodal-TeV-deep-(-6.12 mm)-TT21.5 ////// 467/878 completed  ////

1848-679-2-2-Crossmodal-Auditory-deep-(-5.52 mm)-TT20.1 ////// 554/878 completed  //////  0 min 3.49 sec
1850-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.2 ////// 555/878 completed  //////  0 min 4.70 sec
1851-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.3 ////// 556/878 completed  //////  0 min 3.63 sec
1854-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.6 ////// 557/878 completed  //////  0 min 3.65 sec
1857-679-3-3-Crossmodal-Visual-deep-(-6.36 mm)-TT1.1 ////// 559/878 completed  //////  0 min 3.73 sec
1858-679-3-3-Crossmodal-Visual-deep-(-6.24 mm)-TT2.1 ////// 560/878 completed  //////  0 min 4.94 sec
1859-679-3-3-Crossmodal-Visual-deep-(-6.24 mm)-TT2.2 ////// 561/878 completed  //////  0 min 3.86 sec
1861-679-3-3-Crossmodal-Visual-deep-(-4.56 mm)-TT11.1 ////// 563/878 completed  //////  0 min 3.63 sec
1862-679-3-3-Crossmodal-PER-deep-(-4.8 mm)-TT12.1 ////// 564/878 completed  //////  0 min 3.59 sec
1863-679-3-3-Crossmodal-PER-deep-(-4.8 mm)-TT12.2 ////// 565/878 complet

1997-679-6-6-Crossmodal-Auditory-deep-(-5.28 mm)-TT17.3 ////// 675/878 completed  //////  0 min 4.25 sec
1999-679-6-6-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.1 ////// 676/878 completed  //////  0 min 4.52 sec
2000-679-6-6-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.2 ////// 677/878 completed  //////  0 min 4.98 sec
2001-679-6-6-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.3 ////// 678/878 completed  //////  0 min 4.52 sec
2002-679-6-6-Crossmodal-Auditory-deep-(-5.52 mm)-TT20.1 ////// 679/878 completed  //////  0 min 4.23 sec
2003-679-6-6-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.1 ////// 680/878 completed  //////  0 min 5.31 sec
2004-679-6-6-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.2 ////// 681/878 completed  //////  0 min 4.56 sec
2005-679-6-6-Crossmodal-Auditory-deep-(-6.64 mm)-TT22.1 ////// 682/878 completed  //////  0 min 5.01 sec
2183-699-1-1-Crossmodal-Visual-deep-(-6.96 mm)-TT1.3 ////// 684/878 completed  //////  0 min 5.12 sec
2185-699-1-1-Crossmodal-Visual-deep-(-6.36 mm)-TT3.1 ////// 6

In [ ]:
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id, rat_id, session_id, region = int(cell_info[0]), cell_info[1], cell_info[2], cell_info[5]
    
    # load cell data
    df = pd.read_csv(cell_path/cell_name)
    df.drop(df[df.Correctness==0].index,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df[['Visual','Auditory']] = df[['Visual','Auditory']].fillna('no')
    
    boy_goal = df.loc[df['Visual']=='Boy','RWD_Loc'].values[0]
    boy_aud = df.loc[df['RWD_Loc']==boy_goal,'Auditory'].values[0]
    
    egg_goal = df.loc[df['Visual']=='Egg','RWD_Loc'].values[0]
    egg_aud = df.loc[df['RWD_Loc']==egg_goal,'Auditory'].values[0]
    
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    
    # load regression results
    beta_coef = np.array(f[f'{cell_id}/basic/beta_coef'])
    pval = np.array(f[f'{cell_id}/basic/pval'])
    
    pval_log = -1*np.log(pval)

In [ ]:
f.close()
#s.close()
print('END')